## Important links

* https://nasa-impact.github.io/etci2021/
* https://competitions.codalab.org/competitions/30440

## Data Collection

In [ ]:
!ls -lh train | head -10

In [ ]:
!ls -lh train/bangladesh_20170314t115609/tiles | head -10

In [ ]:
!ls -lh train/bangladesh_20170314t115609/tiles/flood_label | head -10

In [ ]:
!ls -lh train/bangladesh_20170314t115609/tiles/vh | head -10

From [here](https://nasa-impact.github.io/etci2021/#semantic-labels): 

> The provided training data is split across 29 root folders named \<region>\_\<datetime>*, region being the region and datetime being the date and time of the flood event. Each root folder includes 4 sub-folders: vv, vh, water_body_label and flood_label with 2,068 files each. vv and vh correspond to the satellite images listed earlier and images in the flood_label and water_body_label folder provide reference ground truth.

## Imports

In [ ]:
from imutils import paths
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import cv2
import re
import os

import warnings
warnings.filterwarnings("ignore")

## Investigation

In [ ]:
all_image_paths = list(paths.list_images("train"))
print(f"Total images: {int(len(all_image_paths)/2)}")

So, we have 33,406 satellite images and the rest are binary segmentation maps. 

For a given image id (e.g. `nebraska_20170309t002110`), its correspnding ground-truths i.e. the segmentation maps are present in either of these two folders: `water_body_label` and `flood_label`. Let's write a few utility functions for knowing the dataset in a better way. 

**How many unique image IDs are there?**

In [ ]:
image_ids = {path.split("/")[1] for path in all_image_paths}
print(len(image_ids))

Now, let's investigate how are these IDs distributed? **Do all the IDs have the same amount of images present?**

In [ ]:
def get_image_paths(image_id):
    flood_image_root = os.path.join("train", image_id, "tiles", "flood_label")
    water_body_root = os.path.join("train", image_id, "tiles", "water_body_label")
    vh_root = os.path.join("train", image_id, "tiles", "vh")
    vv_root = os.path.join("train", image_id, "tiles", "vv")

    flood_image_paths = list(paths.list_images(flood_image_root))
    water_body_paths = list(paths.list_images(water_body_root))
    vh_image_paths = list(paths.list_images(vh_root))
    vv_image_paths = list(paths.list_images(vv_root))

    return flood_image_paths, water_body_paths,\
        vh_image_paths, vv_image_paths

In [ ]:
distribution_dict = {}

for id in tqdm(image_ids):
    distribution_dict[id] = {}
    flood_image_paths, water_body_paths, vh_image_paths, vv_image_paths = \
        get_image_paths(id)

    distribution_dict[id]["flood_images"] = len(flood_image_paths)
    distribution_dict[id]["water_body_images"] = len(water_body_paths)
    distribution_dict[id]["vh_images"] = len(vh_image_paths)
    distribution_dict[id]["vv_images"] = len(vv_image_paths)

distribution_df = pd.DataFrame.from_dict(distribution_dict).T
assert len(distribution_df) == len(image_ids)
distribution_df

No huge distribution skews noticed. But for **`bangladesh_20170314t115609`** there is a mismatch between the number of flood image maps and the number of VV images.

## Visualization

Now, let's write a utility function that would return the images belonging to the format - `<region>_<datetime>*_x-*_y-*.png`. 

It seems like the VV images should be used for predicting flood levels and VH images should be used for predicting water body levels.

<p align="center">
<img src=https://i.ibb.co/mCZp6X4/image.png></ing>
</p>

However, 

> We expect participants to provide a binary segmentation of the region of interest (ROI), (i.e. 256x256 pixels) as a numpy array with the byte (uint8) data type:
**1: Flood region, 0: Not flood region**.

In [ ]:
# https://stackoverflow.com/a/2669120/7636462
def sorted_nicely(l): 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
all_image_paths = sorted_nicely(all_image_paths)

vv_image_paths = [path for path in all_image_paths if ("vv" in path) and ("ipynb_checkpoints" not in path)]
flood_image_paths = [path for path in all_image_paths if ("flood" in path) and ("ipynb_checkpoints" not in path)]
vh_image_paths = [path for path in all_image_paths if ("vh" in path) and ("ipynb_checkpoints" not in path)]
water_body_label_paths = [path for path in all_image_paths if ("water" in path) and ("ipynb_checkpoints" not in path)]

len(flood_image_paths), len(vv_image_paths), len(vh_image_paths), len(water_body_label_paths)

In [ ]:
all_image_paths[0]

What is `.ipynb_checkpoints` doing here? 😨

In [ ]:
# Verify if we have maintained the order
flood_image_paths[:5], vv_image_paths[:5]

In [ ]:
water_body_label_paths[:5], vh_image_paths[:5]

In [ ]:
def get_image_id(filename):
    return filename.split("/")[1]

In [ ]:
def show_all_four_images(filenames, titles):
    plt.figure(figsize=(20, 10))
    images = []
    for filename in filenames:
        images.append(mpimg.imread(filename))
        
    plt.suptitle(get_image_id(filenames[0]), size=16)
    columns = 4
    
    for i, image in enumerate(images):
        ax = plt.subplot(len(images)/ columns + 1, columns, i + 1)
        ax.set_title(titles[i])
        plt.imshow(image)

    plt.show()

In [ ]:
regex = r"_x-\d+_y-\d+"
compiler = re.compile(regex)

def get_intensity(path):
    return compiler.search(path).group()

In [ ]:
import random

titles = ["V V","V H" , "Land or water before flood/Water body image" ,"After Flood/flood image"]

random_index =  random.sample(range(0, len(vv_image_paths)), 10) 
for i in random_index:
    # The assertions make sure we are operating on the right pairs
    assert  get_intensity(vv_image_paths[i]) == get_intensity(flood_image_paths[i])
    assert  get_intensity(vh_image_paths[i]) == get_intensity(water_body_label_paths[i])
    show_all_four_images([vv_image_paths[i], vh_image_paths[i],  
                          water_body_label_paths[i], flood_image_paths[i] ] , titles)

**Some noise found (from an earlier iteration)**:

* https://ibb.co/m6x9f1S
* https://ibb.co/rfWtJy7

How in an all-white image, any segmentation map is present? 

### Displaying the RGB composite

From [here](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/product-overview/polarimetry):

> The composite RGB (colour) image on the right was created using the VV channel for red, VH channel for green and the ratio $|VV| / |VH|$ for blue.

In [ ]:
from PIL import Image

def show_all_combined_images(i, titles):
    columns = 3

    red, _ , _  = Image.open(vv_image_paths[i]).split()
    red = np.asarray(red)
    _, green, _  = Image.open(vh_image_paths[i]).split()
    green = np.asarray(green)

    blue = np.abs(red) / np.abs(green) 
    blue = (blue * 255).astype(np.uint8)
    rgb = Image.fromarray(np.dstack((red,green,blue)))

    images = [rgb]
    images.append(mpimg.imread(water_body_label_paths[i]))
    images.append(mpimg.imread(flood_image_paths[i]))

    plt.figure(figsize=(20, 10))
    plt.suptitle(get_image_id(vv_image_paths[i]), size=16)
    for i, image in enumerate(images):
        ax = plt.subplot(len(images)/ columns + 1, columns, i + 1)
        ax.set_title(titles[i])
        plt.imshow(image)


titles = ["Combined" , "Land or water before flood/Water body image" ,"After Flood/flood image"]
for i in random_index:
    show_all_combined_images(i , titles)

## Observations

* We need to be careful about the way we would shuffle the samples. We likely wouldn't want to just randomly shuffle them. Because if we do so then the continual order of samples for a particular region and timestamp would get broken. 
* We also cannot randomly sample data points for our local validation set. It's much like predicting the next frame for a given sequence of frames. We would want to train models on a sequence of *ordered* frames and use that to infer the next one. 
* Can we simply discard the blank images (all white ones under `Combined` and their respective labels)? I don't see any point in keeping them. 

## Some preprocessing 

Referred from this [video](https://youtu.be/derOXkPCH80). A PDF is present [here](http://step.esa.int/docs/tutorials/S1TBX%20SAR%20Basics%20Tutorial.pdf). 

### Speckle removal

In [ ]:
# https://stackoverflow.com/a/39786527/7636462
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

def lee_filter(img, size=20):
    img_mean = uniform_filter(img, (size, size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

In [ ]:
random_index =  random.sample(range(0, len(vv_image_paths)), 10) 

In [ ]:
# With Speckle Removal

def show_all_four_images(filenames, titles, speckle=False):
    plt.figure(figsize=(20, 10))
    images = []
    for filename in filenames:
        image = mpimg.imread(filename)
        if speckle:
            lee_filter(image)
        images.append(image)
        
    plt.suptitle(get_image_id(filenames[0]), size=16)
    columns = 4
    
    for i, image in enumerate(images):
        ax = plt.subplot(len(images)/ columns + 1, columns, i + 1)
        ax.set_title(titles[i])
        plt.imshow(image)

    plt.show()

titles = ["V V","V H" , "Land or water before flood/Water body image" ,"After Flood/flood image"]

for i in random_index:
    show_all_four_images([vv_image_paths[i], vh_image_paths[i],  
                          water_body_label_paths[i], flood_image_paths[i] ] , titles, True)

In [ ]:
# Without Speckle

for i in random_index:
    show_all_four_images([vv_image_paths[i], vh_image_paths[i],  
                          water_body_label_paths[i], flood_image_paths[i] ] , titles, False)

Seems like the Sentinel-1 images have gone through some speckle removal already. We can confirm this by examining the distribution of the histograms. 